# ノック41

ノック41 - 50では、決定木を用いて退会会員の予測をしていく。

In [1]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta

In [2]:
# データの読み込み

customer = pd.read_csv("customer_join.csv")
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [3]:
uselog_months = pd.read_csv("use_log_months.csv")
uselog_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


今回は1カ月前のデータから当月の退会予測を導き出す。  

In [4]:
year_months = list(uselog_months["年月"].unique())
year_months

[201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903]

In [5]:
uselog = pd.DataFrame()
for i in range(1,len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"] == year_months[i]]
    tmp.rename(columns = {"count" : "count_0"}, inplace = True)
    tmp_before = uselog_months.loc[uselog_months["年月"] == year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns = {"count" : "count_1"}, inplace = True)
    tmp2 = pd.merge(tmp, tmp_before, on  = "customer_id")
    uselog = pd.concat([uselog,tmp2], ignore_index = True) 
uselog

C:\Users\Takeshi\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4
1,201805,AS009373,4,3
2,201805,AS015315,3,6
3,201805,AS015739,5,7
4,201805,AS015746,1,4
...,...,...,...,...
32645,201903,TS995853,8,11
32646,201903,TS998593,8,7
32647,201903,TS999079,3,2
32648,201903,TS999231,6,6


# ノック42

退会顧客が退会申請をした前月のデータを集計する。  
  
これは、退会申請をした1カ月後にend_dateが登録されるため。  
  
これで退会者の特徴を抽出する。  
Ex. end_dateが2018年8月の顧客は、2018年7月に退会申請をしている。そのため、その顧客の2018年6月のデータを見る。

In [6]:
exit_customer = customer.loc[customer["is_deleted"] == 1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months = 1)
exit_customer["exit_date"] = pd.to_datetime(exit_customer["exit_date"])
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on = ["customer_id", "年月"], how = "left")
print(len(exit_uselog))
exit_uselog.head()

C:\Users\Takeshi\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Takeshi\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Takeshi\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

32650


C:\Users\Takeshi\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Takeshi\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015315,3,6,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015739,5,7,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015746,1,4,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [7]:
exit_uselog = exit_uselog.dropna(subset = ["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))

1052
1052


# ノック43

次に、継続顧客に関してのデータを集計していく。  
まずは、継続顧客のみのデータとuselogを結合する。

In [8]:
customer_continue = customer.loc[customer["is_deleted"] == 0]
uselog_customer_continue = pd.merge(uselog, customer_continue, on = "customer_id")
uselog_customer_continue

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201805,AS002855,5,4,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.5,5.0,7,2,1,2019-04-30,29
1,201806,AS002855,5,5,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.5,5.0,7,2,1,2019-04-30,29
2,201807,AS002855,5,5,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.5,5.0,7,2,1,2019-04-30,29
3,201808,AS002855,3,5,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.5,5.0,7,2,1,2019-04-30,29
4,201809,AS002855,7,3,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.5,5.0,7,2,1,2019-04-30,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26729,201903,TS613765,7,8,XXXXX,C02,F,2019-02-11,NaN,CA1,...,デイタイム,7500,通常,7.5,7.5,8,7,1,2019-04-30,2
26730,201903,TS614788,9,9,XXXXX,C03,M,2019-02-07,NaN,CA1,...,ナイト,6000,通常,9.0,9.0,9,9,1,2019-04-30,2
26731,201903,TS860472,6,6,XXXX,C02,F,2019-02-02,NaN,CA1,...,デイタイム,7500,通常,6.0,6.0,6,6,1,2019-04-30,2
26732,201903,TS878897,7,12,XXXX,C01,M,2019-02-05,NaN,CA1,...,オールタイム,10500,通常,9.5,9.5,12,7,1,2019-04-30,2


退会顧客情報が1052列に対して継続顧客情報が26734列なので、数を合少しでも合わせたい。  
そこで、複数の行がある同一顧客情報からランダムに一つの月の情報を抽出する。

In [9]:
unique_id = list(uselog_customer_continue["customer_id"].unique())
customer_continue_sample = pd.DataFrame()
for i in range(len(unique_id)):
    customer_randomsample = uselog_customer_continue.loc[uselog_customer_continue["customer_id"] == unique_id[i]].sample(n = 1)
    customer_continue_sample = pd.concat([customer_continue_sample, customer_randomsample], axis = 0)
customer_continue_sample.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
6,201811,AS002855,5,3,XXXX,C03,F,2016-11-01,NaN,CA1,...,ナイト,6000,通常,4.500000,5.0,7,2,1,2019-04-30,29
21,201903,AS009373,5,6,XX,C01,F,2015-11-01,NaN,CA1,...,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,41
23,201806,AS015315,3,3,XXXXX,C01,M,2015-07-01,NaN,CA1,...,オールタイム,10500,通常,4.833333,5.0,7,3,1,2019-04-30,45
37,201809,AS015739,6,5,XXXXX,C03,M,2017-06-01,NaN,CA1,...,ナイト,6000,通常,5.583333,5.5,8,4,1,2019-04-30,22
45,201806,AS019860,6,8,XXXXXX,C03,F,2017-10-01,NaN,CA1,...,ナイト,6000,通常,6.083333,6.0,9,4,1,2019-04-30,18


In [10]:
# 抽出がうまく出来ているか確認

print(len(unique_id))
print(len(customer_continue_sample["customer_id"].unique()))
print(customer_continue_sample["年月"].value_counts())

2777
2777
201903    410
201902    319
201901    296
201810    238
201811    231
201806    228
201812    221
201809    216
201808    211
201807    207
201805    200
Name: 年月, dtype: int64


2777列まで減らせた事と、ランダムに各月から均等に抽出出来ている事がわかる。  
  

ここで、継続顧客データと退会顧客データを結合する。

In [11]:
customer_continue_exit = pd.concat([exit_uselog,customer_continue_sample], axis = 0).reset_index()
customer_continue_exit

,index,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,16,201805,AS055680,3,3,XXXXX,C01,M,2018-03-01,2018-06-30 00:00:00,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
1,48,201805,AS169823,2,3,XX,C01,M,2017-11-01,2018-06-30 00:00:00,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
2,98,201805,AS305860,5,3,XXXX,C01,M,2017-06-01,2018-06-30 00:00:00,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
3,116,201805,AS363699,5,3,XXXXX,C01,M,2018-02-01,2018-06-30 00:00:00,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
4,133,201805,AS417696,1,4,XX,C03,F,2017-09-01,2018-06-30 00:00:00,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3824,26729,201903,TS613765,7,8,XXXXX,C02,F,2019-02-11,NaN,...,7500.0,通常,7.500000,7.5,8.0,7.0,1.0,2019-04-30,2.0,NaT
3825,26730,201903,TS614788,9,9,XXXXX,C03,M,2019-02-07,NaN,...,6000.0,通常,9.000000,9.0,9.0,9.0,1.0,2019-04-30,2.0,NaT
3826,26731,201903,TS860472,6,6,XXXX,C02,F,2019-02-02,NaN,...,7500.0,通常,6.000000,6.0,6.0,6.0,1.0,2019-04-30,2.0,NaT
3827,26732,201903,TS878897,7,12,XXXX,C01,M,2019-02-05,NaN,...,10500.0,通常,9.500000,9.5,12.0,7.0,1.0,2019-04-30,2.0,NaT


# ノック44

各顧客の在籍期間の列を追加する。

まず、継続顧客は2019年4月30日をend_dateとして処理する。

In [12]:
customer_continue_exit.loc[customer_continue_exit["end_date"].isnull(),"end_date"] = datetime.datetime(2019, 4, 30,0,0,0)

次に、end_date/start_dateを日付型に変更。  
その後、start_dateとの差を計算して、列を作成。  
最後に年月に変換。

In [13]:
customer_continue_exit["end_date"] = pd.to_datetime(customer_continue_exit["end_date"])
customer_continue_exit["start_date"] = pd.to_datetime(customer_continue_exit["start_date"])

In [16]:
customer_continue_exit["mem_period"] = None
for i in range(len(customer_continue_exit)):
    delta = relativedelta(customer_continue_exit.iloc[i,9],customer_continue_exit.iloc[i,8])
    customer_continue_exit.iloc[i, 23] = delta.years * 12 + delta.months
customer_continue_exit["mem_period"]    

0        3
1        7
2       12
3        4
4        9
        ..
3824     2
3825     2
3826     2
3827     2
3828     2
Name: mem_period, Length: 3829, dtype: object

# ノック45

全体の欠損値を調べる。

In [17]:
customer_continue_exit.isnull().sum()

index                   0
年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date                0
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2777
mem_period              0
dtype: int64

exit_dateにのみ欠損値が含まれている。  
end_dateに含まれていないのは、欠損値に2019年4月30日を代入したため。

# ノック46(ここから自身の解答)

機械学習で用いる特徴量に絞りこむ。

In [18]:
for_ml = customer_continue_exit[["campaign_name","class_name","gender","count_1","routine_flg","mem_period","is_deleted"]]
for_ml.head()

,campaign_name,class_name,gender,count_1,routine_flg,mem_period,is_deleted
0,通常,オールタイム,M,3,0.0,3,1.0
1,通常,オールタイム,M,3,1.0,7,1.0
2,通常,オールタイム,M,3,0.0,12,1.0
3,通常,オールタイム,M,3,0.0,4,1.0
4,通常,ナイト,F,4,0.0,9,1.0


mem_periodをint型に変換する。

In [19]:
for_ml["mem_period"] = for_ml["mem_period"].astype(int)

C:\Users\Takeshi\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


カテゴリカル変数をダミー変数に変換する。

In [20]:
for_ml_dummy = pd.get_dummies(for_ml)
for_ml_dummy.head()

,count_1,routine_flg,mem_period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,3,0.0,3,1.0,0,0,1,1,0,0,0,1
1,3,1.0,7,1.0,0,0,1,1,0,0,0,1
2,3,0.0,12,1.0,0,0,1,1,0,0,0,1
3,3,0.0,4,1.0,0,0,1,1,0,0,0,1
4,4,0.0,9,1.0,0,0,1,0,0,1,1,0


ここで、以下の三つの列を削除する。  
campaign_name_通常  
class_name_ナイト  	
gender_M  
これは、それぞれのカテゴリーでの他の要素によって特定されるため。  
Ex. gender_Fが0であればgender_Mは必ず1になる。

In [21]:
for_ml_dummy

,count_1,routine_flg,mem_period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,3,0.0,3,1.0,0,0,1,1,0,0,0,1
1,3,1.0,7,1.0,0,0,1,1,0,0,0,1
2,3,0.0,12,1.0,0,0,1,1,0,0,0,1
3,3,0.0,4,1.0,0,0,1,1,0,0,0,1
4,4,0.0,9,1.0,0,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3824,8,1.0,2,0.0,0,0,1,0,1,0,1,0
3825,9,1.0,2,0.0,0,0,1,0,0,1,0,1
3826,6,1.0,2,0.0,0,0,1,0,1,0,1,0
3827,12,1.0,2,0.0,0,0,1,1,0,0,0,1


In [23]:
for_ml_dummy.drop(["campaign_name_通常","class_name_ナイト", "gender_M"], axis = 1, inplace = True)
for_ml_dummy

,count_1,routine_flg,mem_period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,3,0.0,3,1.0,0,0,1,0,0
1,3,1.0,7,1.0,0,0,1,0,0
2,3,0.0,12,1.0,0,0,1,0,0
3,3,0.0,4,1.0,0,0,1,0,0
4,4,0.0,9,1.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
3824,8,1.0,2,0.0,0,0,0,1,1
3825,9,1.0,2,0.0,0,0,0,0,0
3826,6,1.0,2,0.0,0,0,0,1,1
3827,12,1.0,2,0.0,0,0,1,0,0
